In [8]:
# Import all the dependencies
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores.qdrant import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from peft import PeftModel, PeftConfig
import ctranslate2
#
from tqdm.auto import tqdm
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd

#
import os
import random
import torch
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the document that you need to parse, please change the location to where the pdf resides

# Load 1 PDF file
# loader = PyPDFLoader("/mnt/data/smuckers_poc/RAG/2024-First-Quarter-Results.pdf")
# or load an entire folder
loader = PyPDFDirectoryLoader("/mnt/data/Local-RAG-Custom/nissan")
data = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [4]:
print(f"There are {len(data)} chunks in the documents")

There are 1975 chunks in the documents


In [5]:
# Pick a sample page
print(data[random.randint(0, len(data))])

page_content='detailed explanations of the vehicle fea-\ntures and operation.\nCHARGING THE LI-ION BATTERY\nWARNING\nThe EV (Electric Vehicle) system uses\na high voltage current. Failure to\nfollow the proper handling instruc-\ntions may cause serious injury or\ndeath. Be sure to read the “CH.\nCharging” section and follow the\nprocedures and guidelines de-scribed.\nCHECKING LI-ION BATTERY CHAR-\nGING STATUS\nThe Li-ion battery charge status can be\nchecked on the NissanConnect® Services\napp (if so equipped).\nIf the Li-ion battery is not sufficiently\ncharged, you can start charging the Li-\nion battery via the remote charge func-\ntion. For additional information, see\n“Charging related remote function” (P.CH-37).LIFE WITH AN EV (Electric Vehicle)\n(scene guide)' metadata={'source': '/mnt/data/Local-RAG-Custom/nissan/2023-nissan-ariya-owner-manual.pdf', 'page': 27}


In [6]:
# Split the data into pages
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

1975 1975


In [11]:
# Setup the prompt template to use for the QA bot
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#


In [12]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/Local-RAG-Custom/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [21]:
# Uncomment this code if you want to store the embeddings in Qdrant in-memory
# doc_store = Qdrant.from_texts(texts,
#                              metadatas=metadatas,
#                              embedding=embeddings,
#                              location=":memory:",
#                              collection=f"{embedding_model_name}_press_release")

In [23]:
 # Persist the embeddings to disk
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              path="/mnt/artifacts/notebook/local_qdrant/",
                              prefer_grpc=True,
                              collection=f"{embedding_model_name}_press_release")

In [15]:
# Load the model and the tokenizer
chain_type_kwargs = {"prompt": PROMPT}


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_id = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir="/mnt/data/Local-RAG-Custom/model_cache/",
    quantization_config=bnb_config,
    device_map='auto'
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = model.config.eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


('/mnt/data/Local-RAG-Custom/model_cache/tokenizer_config.json',
 '/mnt/data/Local-RAG-Custom/model_cache/special_tokens_map.json',
 '/mnt/data/Local-RAG-Custom/model_cache/tokenizer.model',
 '/mnt/data/Local-RAG-Custom/model_cache/added_tokens.json',
 '/mnt/data/Local-RAG-Custom/model_cache/tokenizer.json')

In [25]:
# Setup the QA chain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
rag_llm = HuggingFacePipeline(pipeline=pipe)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [27]:
# Ask a question
user_question = input("Please provide your question here :")
result = qa_chain(user_question)
print(result['result'])

Please provide your question here : what is the tyre pressure?


The tyre pressure for a Nissan Ariya is 30 psi (200 kPa) for the front tires and 26 psi (180 kPa) for the rear tires, according to the Tire and Loading Information label on the vehicle.

QUOTE:
"The tire pressurizes as tire temperature rises." (WBI0031X)

Please let me know if you need any further assistance.


In [13]:
print(torch.cuda.memory_allocated())

0


In [26]:
del model

NameError: name 'model' is not defined

In [ ]:
print(torch.cuda.memory_allocated())

In [16]:
# full_model_path = 'subirmansukhani/llama-2-7b-miniguanaco'
full_model_path = '/mnt/data/Local-RAG-Custom/model_cache/models--NousResearch--Llama-2-7b-chat-hf/snapshots/37892f30c23786c0d5367d80481fa0d9fba93cf8'
# Copy the file over from this path to /mnt/data/ or wherever your Domino datasets is if you intend to use this model in an app
tokenizer.save_pretrained(full_model_path)
ct2_path ='/mnt/data/Local-RAG-Custom/llama2/llama2-ct/'
quantization ='float16'

os.system(f"sudo ct2-transformers-converter --model {full_model_path} --copy_files tokenizer.model --output_dir {ct2_path} --quantization {quantization}  --low_cpu_mem_usage")

print(" Model Converted successfully")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.79it/s]
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Speci

 Model Converted successfully


In [10]:
model_path = '/mnt/data/Local-RAG-Custom/llama2/llama2-ct'
model_id = "NousResearch/Llama-2-7b-chat-hf"
model_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the ctranslate model
generator = ctranslate2.Generator(model_path, device=model_device)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

RuntimeError: CUDA failed with error out of memory

In [ ]:
# Setup the QA chain
pipe = pipeline("text-generation", model=generator, tokenizer=tokenizer, max_new_tokens=200)
rag_llm = HuggingFacePipeline(pipeline=pipe)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )